# K-fold cross validation

In [1]:
using RDatasets
using MLJ

┌ Info: Precompiling MLJ [add582a8-e3ab-11e8-2d5e-e98b27df1bc7]
└ @ Base loading.jl:1423
┌ Warning: Replacing module `CategoricalDistributions`
└ @ Base loading.jl:1036
┌ Warning: Module MLJBase with build ID 106585755292894 is missing from the cache.
│ This may mean MLJBase [a7f614a8-145f-11e9-1d2a-a57a1082229d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for MLJ [add582a8-e3ab-11e8-2d5e-e98b27df1bc7]
│   exception = ErrorException("Required dependency MLJTuning [03970b2e-30c4-11ea-3135-d1576263f10f] failed to load from a cache file.")
└ @ Base loading.jl:1132
┌ Warning: Replacing module `CategoricalDistributions`
└ @ Base loading.jl:1036
┌ Info: Precompiling MLJIteration [614be32b-d00c-4edb-bd02-1eb411ab5e55]
└ @ Base loading.jl:1423
┌ Warning: Module MLJBase with build ID 106585755292894 is missing from the cache.
│ This may mean MLJBase [a7f614a8-

In [2]:
iris = dataset("datasets", "iris")
first(iris, 10)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [3]:
y, X = unpack(iris, ==(:Species), colname -> true);
first(X, 6)

,SepalLength,SepalWidth,PetalLength,PetalWidth
,Float64,Float64,Float64,Float64
1,5.1,3.5,1.4,0.2
2,4.9,3.0,1.4,0.2
3,4.7,3.2,1.3,0.2
4,4.6,3.1,1.5,0.2
5,5.0,3.6,1.4,0.2
6,5.4,3.9,1.7,0.4


### Model

In [4]:
DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/yuehhua/.julia/packages/MLJModels/lDzCR/src/loading.jl:168


MLJDecisionTreeInterface.DecisionTreeClassifier

## k-fold cv

In [5]:
cv = CV(nfolds=5)

CV(
  nfolds = 5, 
  shuffle = false, 
  rng = Random._GLOBAL_RNG())

In [6]:
tree = evaluate(DecisionTreeClassifier(), X, y, resampling=cv, measure=accuracy)

Evaluating over 5 folds: 100%[=========================] Time: 0:00:07


PerformanceEvaluation object with these fields:
  measure, operation, measurement, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_rows
Extract:
┌────────────┬──────────────┬─────────────┬─────────┬───────────────────────────
│ measure    │ operation    │ measurement │ 1.96*SE │ per_fold                 ⋯
├────────────┼──────────────┼─────────────┼─────────┼───────────────────────────
│ Accuracy() │ predict_mode │ 0.92        │ 0.0941  │ [1.0, 1.0, 0.9, 0.933, 0 ⋯
└────────────┴──────────────┴─────────────┴─────────┴───────────────────────────
                                                                1 column omitted


In [7]:
tree = machine(DecisionTreeClassifier(), X, y)
evaluate!(tree, resampling=cv, measure=accuracy)

PerformanceEvaluation object with these fields:
  measure, operation, measurement, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_rows
Extract:
┌────────────┬──────────────┬─────────────┬─────────┬───────────────────────────
│ measure    │ operation    │ measurement │ 1.96*SE │ per_fold                 ⋯
├────────────┼──────────────┼─────────────┼─────────┼───────────────────────────
│ Accuracy() │ predict_mode │ 0.9         │ 0.0952  │ [1.0, 0.967, 0.833, 0.93 ⋯
└────────────┴──────────────┴─────────────┴─────────┴───────────────────────────
                                                                1 column omitted


In [8]:
tree

Machine trained 5 times; caches data
  model: DecisionTreeClassifier(max_depth = -1, …)
  args: 
    1:	Source @323 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @796 ⏎ `AbstractVector{Multiclass{3}}`
